In [23]:
import os, sys, argparse, paramiko
from getpass import getpass
from os.path import exists

In [24]:
# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password:  ")

# Host information
host = "192.168.6.71"
port = 2222
username = "sarah.fornaldes"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication failed")

# TASK 2

In [25]:
sftp = ssh.open_sftp()

# Run a command to scan all known directories on Linux that have a *bin* directory
command = 'sudo -S /home/ubuntu/kraken --folder /usr/bin --folder --folder /usr/sbin/ --folder /usr/local/bin --folder /sbin --folder /usr/local/sbin --folder /bin'

ssh.exec_command('kraken')
sftp.put('kraken', 'kraken')

thePass = getpass(prompt="Please enter your SSH password:  ")

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

stdin.write(thePass + "\n")
stdin.flush()

# Get results  from stdout
lines =  stdout.readlines()
#print(lines)

# Convert the list to a string
output =  ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output = sepHeader + output + sepFooter

# Save the cmd_output to a file
with open('results123.txt', 'a') as f:
    f.write(cmd_output)

#ssh.close()
#sftp.close()

# TASK 3

In [26]:
#sftp = ssh.open_sftp()

# Uploads results.txt file
uploadFiles = ['results123.txt']

# Runs the lsof command, lsof lists all open files for a process                
command = "sudo -S lsof -i -n | awk '{print $2}' "

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

# Input password to run the command as sudo, including a new line character that provides "Enter" as if you were typing it in on a keyboard
stdin.write(thePass + "\n")

# Get results  from stdout
lines =  stdout.readlines()

# Convert the list to a string
output =  ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output =  sepHeader + output +  sepFooter

# Save the cmd_output to a file
with open('resultsLsof.txt', 'a') as f:
    f.write(cmd_output)

# End SFTP Session
#ssh.close()
#sftp.close()

In [27]:
# Runs the lsof command, lsof lists all open files for a process                
command = ['sudo -S lsof -p  519423','sudo -S lsof -p  561559'] 

for eachCMD in command:

    stdin, stdout, stderr  = ssh.exec_command(eachCMD)
    stdin.write(thePass + "\n")
    lines =  stdout.readlines()
    output =  ''.join(lines)
    sepHeader = '' + '### BEGIN ' + eachCMD + ' ###\n\n'
    sepFooter = '' + '### END ' + eachCMD + ' ###\n\n'
    cmd_output =  sepHeader + output +  sepFooter

# Save the cmd_output to a file
with open('resultsLsof-p.txt', 'a') as f:
    f.write(cmd_output)

# End SFTP Session
#ssh.close()
#sftp.close()

In [28]:
#GETS THE NAME COLUMN FROM THE PIDS

command = "sudo -S lsof -p 561559  | awk '{print $9}' "
stdin, stdout, stderr  = ssh.exec_command(command)
stdin.write(thePass + "\n")
lines =  stdout.readlines()
output =  ''.join(lines)
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'
sepFooter = '' + '### END ' + command + ' ###\n\n'
cmd_output =  sepHeader + output +  sepFooter

# Save the cmd_output to a file
with open('resultsLsof-pid.txt', 'a') as f:
    f.write(cmd_output)

# TASK 4

In [29]:
command = 'ls -l /home/p.dalton'
stdin, stdout, stderr  = ssh.exec_command(command)
for line in stdout.readlines():
    print(line)


total 508

-rwxr-xr-x 1 p.dalton p.dalton 511476 Feb 18  2022 aqwerpuqwerj

-rw-r--r-- 1 p.dalton p.dalton      6 Oct 25 23:32 d.pid

drwxr-xr-x 2 p.dalton p.dalton   4096 Oct 29 19:51 test-lk



In [30]:
# PUTTY'd into 192.168.6.71, cd /home/p.dalton, cd test-lk, ls
sftp.get('/home/p.dalton/aqwerpuqwerj','aqwerpuqwerj')
sftp.get('/home/p.dalton/d.pid','d.pid')
sftp.get('/home/p.dalton/test-lk/d.pid','d.pid')

# TASK 5

In [31]:
# What is the process name? egrep
command = 'ps -p 561559 -o comm='
stdin, stdout, stderr  = ssh.exec_command(command)
for line in stdout.readlines():
    print(line)

egrep



In [32]:
# Which user is running the process? p.dalton
command = 'ps -aux | grep 561559 '
stdin, stdout, stderr  = ssh.exec_command(command)
for line in stdout.readlines():
    print(line)


p.dalton  561559  0.0  0.0 1003532 1212 ?        Ssl  Oct25   0:01 [go-daemon sample]

sarah.f+  633122  0.0  0.0   8616  3064 ?        Ss   19:56   0:00 bash -c ps -aux | grep 561559 

sarah.f+  633124  0.0  0.0   8160   720 ?        S    19:56   0:00 grep 561559



In [33]:
# Has the user logged in before via SSH? Yes
# When did the user login? Tue Oct 25 23:23:31
command = 'lastlog | grep p.dalton'
stdin, stdout, stderr  = ssh.exec_command(command)
for line in stdout.readlines():
    print(line)

p.dalton         pts/0    216.93.152.115   Tue Oct 25 23:23:31 -0400 2022



In [34]:
# How long was the login session? 9 seconds
command = 'last p.dalton'
stdin, stdout, stderr  = ssh.exec_command(command)
for line in stdout.readlines():
    print(line)


p.dalton pts/0        216.93.152.115   Tue Oct 25 23:23 - 23:32  (00:09)

p.dalton pts/0        192.168.6.72     Sun Feb 27 00:52 - 01:17  (00:24)

p.dalton pts/1        10.223.0.132     Sat Feb 26 23:50 - 00:27  (00:36)

p.dalton pts/1        10.223.0.132     Sat Feb 26 23:44 - 23:44  (00:00)



wtmp begins Sat Feb 26 23:26:55 2022



In [35]:
# What is located in the unauthorized user’s home directory? The executable file aqwerpuqwerj, the file d.pid, and the directory test-lk. In test-lk there is a executable file called d.pid
command = 'ls -l /home/p.dalton'
stdin, stdout, stderr  = ssh.exec_command(command)
for line in stdout.readlines():
    print(line)


total 508

-rwxr-xr-x 1 p.dalton p.dalton 511476 Feb 18  2022 aqwerpuqwerj

-rw-r--r-- 1 p.dalton p.dalton      6 Oct 25 23:32 d.pid

drwxr-xr-x 2 p.dalton p.dalton   4096 Oct 29 19:51 test-lk



In [36]:
# End SFTP Session
ssh.close()
sftp.close()

# Reflections

- Based on what you discovered and the information in the threat report, what is your conclusion about the host you analyzed?
    - It has been compromised. An unauthorized user accessed the sytem via ssh on Tue Oct 25 23:23:31 for 9 seconds. The executable file aqwerpuqwerj, the file d.pid, and the directory test-lk. In test-lk there is a executable file called d.pid.

- What did you like the most and least about this assignment?
    - Most: the use of paramiko was fun.
    - Least: it was very hard.

- What additional questions do you have?
    - What resources do you have for us to study.
